In [1]:
import pandas as pd
import import_ipynb
import utils_cdt as cdt
import pyodbc
import time

importing Jupyter notebook from utils_cdt.ipynb


In [2]:
infoDataBases = {"ip":"",
                 "base":""}

In [3]:
#Consulta de BASE SQL Server
arq = open("querys/transacao_nao_cadastrada.sql","r+")
consulta_transacoes = arq.read()
arq.close()

In [4]:
arq = open("querys/duplicidade_codigosprocessamento.sql","r+")
consulta_duplicidade = arq.read()
arq.close()

In [5]:
arq = open("querys/prepago_limite.sql","r+")
consulta_prepago = arq.read()
arq.close()

In [6]:
ini = time.time()
df_transacoes = cdt.extrairDados(infoDataBases, consulta_transacoes, spark=False)
fim = time.time()
print ("Tempo em segundos : ", fim-ini)

Tempo em segundos :  20.307000637054443


In [7]:
ini = time.time()
df_duplicidade = cdt.extrairDados(infoDataBases, consulta_duplicidade, spark=False)
fim = time.time()
print ("Tempo em segundos : ", fim-ini)

Tempo em segundos :  0.8818726539611816


In [8]:
ini = time.time()
df_prepago = cdt.extrairDados(infoDataBases, consulta_prepago, spark=False)
fim = time.time()
print ("Tempo em segundos : ", fim-ini)

Tempo em segundos :  11.055957317352295


In [9]:
#Necessario leitura da base anteriormente
# Inputs: df_transacoes, df_duplicidade, df_prepago
# Outputs: array de dicionario
# {Descricao, DataFrame}

def validacaoCadastralAutorizacao():
    resumo = pd.DataFrame(columns = ['Emissor','Descricao', 'Quantidade'])
    resultado = []
    if(not df_transacoes.empty):
        #Monta dataframe resumido
        linha = {'Emissor': df_transacoes['Emissor'][0], 'Descricao':'Transações negadas por falta de cadastro na vinculos operações', 'Quantidade': df_transacoes['registros'].sum()}
        resumo = resumo.append(linha, ignore_index = True)
        
        #DataFrame do erro
        resultado.append({'Descricao': 'Transações negadas por falta de cadastro na vínculos operações ', 'DataFrame': df_transacoes})

    if(not df_duplicidade.empty):
        #Monta dataframe resumido
        linha = {'Emissor': df_duplicidade['Emissor'][0],'Descricao':'Duplicidade na tabela codigosprocessamento', 'Quantidade': df_duplicidade['registros'].count()}
        resumo = resumo.append(linha, ignore_index = True)
        
        #DataFrame do erro
        resultado.append({'Descricao': 'Registros duplicados na tabela codigosprocessamento ', 'DataFrame': df_duplicidade})
        
    if(not df_prepago.empty):
        linha = {'Emissor': df_prepago['Emissor'][0],'Descricao':'Produtos pré-pagos que não afetam nenhum limite ou que aumentam o limite', 'Quantidade': df_prepago['Emissor'].count()}
        resumo= resumo.append(linha, ignore_index = True)
        
         #DataFrame do erro
        resultado.append({'Descricao': 'Produtos pré-pagos que não afetam limite ', 'DataFrame': df_prepago})
        
    #lista de dicionarios
    #{Descricao: '', DataFrame: }
    resultado.insert(0, {'Descricao':'', 'DataFrame': resumo})
    return resultado           

In [18]:
#retorno lista de dicionarios
resultado = validacaoCadastralAutorizacao()
resultado[2]['Descricao']

'Transações negadas por falta de cadastro na vínculos operações '

In [21]:
resultado

[{'Descricao': '',
  'DataFrame':      Emissor                                          Descricao Quantidade
  0  Pagseguro  Transações negadas por falta de cadastro na vi...      10293
  1  Pagseguro         Duplicidade na tabela codigosprocessamento          1
  2  Pagseguro  Produtos pré-pagos que não afetam nenhum limit...          4},
 {'Descricao': 'Transações negadas por falta de cadastro na vínculos operações ',
  'DataFrame':       Emissor           Descricao_Odin  Id_produto CodigoProcessamento  \
  0   Pagseguro  OPERACAO_NAO_ENCONTRADA           1              003800   
  1   Pagseguro  OPERACAO_NAO_ENCONTRADA           1              011000   
  2   Pagseguro  OPERACAO_NAO_ENCONTRADA           1              012000   
  3   Pagseguro  OPERACAO_NAO_ENCONTRADA           3              003800   
  4   Pagseguro  OPERACAO_NAO_ENCONTRADA           3              003810   
  5   Pagseguro  OPERACAO_NAO_ENCONTRADA           3              011000   
  6   Pagseguro  OPERACAO_NAO_E